In [2]:
import pandas as pd
import numpy as np
import sklearn as skl
import os

In [3]:
PATIENT_PATH = "/nv/vol141/phys_nrf/Emery/dataset/"
BLACKLIST=["MK"]
patient_names=list(pd.read_csv(PATIENT_PATH+"Dates.csv").columns)
for name in BLACKLIST:
    try:
        patient_names.remove(name)
    except:
        print(name,"in blacklist but not in dataset")
X,y=[],[]
for name in patient_names:
    print(name+":",end=' ')
    try:
        ITV=pd.read_csv(PATIENT_PATH+name+"/ITV.csv")
        Random_ITV=pd.read_csv(PATIENT_PATH+name+"/R_ITV.csv")
        print("Loaded",len(ITV.columns)-1,"scans.")
        keys=[]
        for i in ITV.index:
            try:
                test = float(ITV["Pre"][i])
                keys.append(i)
            except:
                pass
        for col in ITV.columns:
            if not col.startswith("Pre"):
                continue
            X.append([float(ITV[col][k]) for k in keys])
            y.append(1)
            X.append([float(Random_ITV[col][k]) for k in keys])
            y.append(0)
    except Exception as e:
        print("Failed.")
print("Loaded",len(X),"scans total.")

BB: Loaded 4 scans.
BE: Loaded 8 scans.
BJ: Loaded 5 scans.
CT: Loaded 2 scans.
DA: Loaded 2 scans.
FJ: Loaded 2 scans.
HD: Loaded 7 scans.
HI: Loaded 2 scans.
MB: Loaded 2 scans.
SB: Loaded 4 scans.
LK: Loaded 7 scans.
LL: Loaded 7 scans.
SE: Loaded 5 scans.
SW: Loaded 2 scans.
DJ: Loaded 2 scans.
SJ: Loaded 5 scans.
HR: Loaded 2 scans.
MJ: Failed.
Loaded 136 scans total.


In [4]:
length=len(X[0])
for i in X:
    if not len(i)==length:
        print("WARNING! Different lengths in X!")
X=np.array(X)
y=np.array(y)
print("Number of features:",len(X[0]))

Number of features: 112


In [5]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X)
fitX=scaler.transform(X)

In [6]:
from sklearn.svm import SVC
import time
#linear_SVM = SVC(kernel='linear', random_state=0)
import random
#fitX,y = [fitX[i] for i in rand], [y[i] for i in rand]



In [7]:
split = 120
trials = 1000
C=0.3
train_results, test_results = [],[]

start=time.perf_counter()
for trial in range(trials):
    linear_SVM = SVC(C=C, kernel='linear', random_state=0)
    train_perf, test_perf = [],[]
    random.seed(trial)
    rand=[i for i in range(len(y))]
    random.shuffle(rand)
    train_X,train_y = [fitX[i] for i in rand], [y[i] for i in rand]
    
    linear_SVM.fit(train_X[:split], train_y[:split])
    
    pred = linear_SVM.predict(train_X)
    for i in range(split):
        train_results.append(int(pred[i]==train_y[i]))
    for i in range(split,len(pred)):
        test_results.append(int(pred[i]==train_y[i]))
end = time.perf_counter()
print("Time:",end-start)
print("Train Accuracy:",np.mean(train_results),"+/-",np.std(train_results)/len(train_results)**0.5)
print("Test Accuracy:",np.mean(test_results),"+/-",np.std(test_results)/len(test_results)**0.5)

Time: 4.895699078217149
Train Accuracy: 0.931025 +/- 0.0007315363363896106
Test Accuracy: 0.8340625 +/- 0.0029411120993357897


In [8]:
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier(n_estimators=200)
randomForest.fit(fitX[:split],y[:split])

RandomForestClassifier(n_estimators=200)

In [102]:
split = 120
trials = 1000
n_trees = 200
train_results, test_results = [],[]

start=time.perf_counter()
for trial in range(trials):
    randomForest = RandomForestClassifier(n_estimators=n_trees, criterion='gini')
    train_perf, test_perf = [],[]
    random.seed(trial)
    rand=[i for i in range(len(y))]
    random.shuffle(rand)
    train_X,train_y = [fitX[i] for i in rand], [y[i] for i in rand]
    
    randomForest.fit(train_X[:split], train_y[:split])
    
    pred = randomForest.predict(train_X)
    for i in range(split):
        train_results.append(int(pred[i]==train_y[i]))
    for i in range(split,len(pred)):
        test_results.append(int(pred[i]==train_y[i]))
end = time.perf_counter()
print("Time:",end-start)
print("Train Accuracy:",np.mean(train_results),"+/-",np.std(train_results)/len(train_results)**0.5)
print("Test Accuracy:",np.mean(test_results),"+/-",np.std(test_results)/len(test_results)**0.5)

Time: 337.43502637185156
Train Accuracy: 1.0 +/- 0.0
Test Accuracy: 0.8586875 +/- 0.0027538981161227397
